# Lesson 1 - Semantic Search

Welcome to Lesson 1. 

To access the `requirement.txt` file, go to `File` and click on `Open`.
 
I hope you enjoy this course!

### Import the Needed Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import DLAIUtils

import os
import time
import torch

In [3]:
from tqdm.auto import tqdm

### Load the Dataset

In [4]:
dataset = load_dataset('quora', split='train[240000:290000]')

In [5]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [6]:
record = dataset['questions'][0]
print(list(set(record['text'])))

['What is the truth of life?', "What's the evil truth of life?"]


In [7]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [9]:
query = 'which city is the most populated in the world?'
xq = model.encode(query)
xq.shape

(384,)

### Setup Pinecone

In [20]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

In [21]:
PINECONE_API_KEY

'67849698-b32a-42f1-b55d-fb4eb8091d3a'

In [22]:
from pinecone import ServerlessSpec, PodSpec

In [27]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name('dl-ml')

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
# pinecone.create_index(name=INDEX_NAME, 
#     dimension=model.get_sentence_embedding_dimension(), 
#     metric='cosine',
#     spec=PodSpec(
#     environment="gcp-starter",
#     pod_type="starter"
#   ))
# index = pinecone.Index(INDEX_NAME)
# print(index)

dl-ml-l5tifdu0t3blbkfjtqg8fo5dwfuxqwaoniaj


### Create Embeddings and Upsert to Pinecone

In [29]:
from pinecone import Pinecone

pc = Pinecone(api_key="67849698-b32a-42f1-b55d-fb4eb8091d3a")
index = pc.Index("dl-app")

In [30]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

100%|██████████| 50/50 [02:30<00:00,  3.01s/it]


In [31]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

### Run Your Query

In [32]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [33]:
run_query('which city has the highest population in the world?')

0.7: Where is the most beautiful city in the world?
0.69: What is the most beautiful city in the world?
0.64: Which city has the most museums per capita?
0.59: Which is the highest peak of the world?
0.58: Which country has the highest per capita income?
0.56: What percentage of the world's population lives in developing countries?
0.55: Which is the highest mountain in the world?
0.54: What is the most remote place in the world?
0.53: Why is Uttar Pradesh the most populous state in India?
0.49: Which are the most dangerous places on the earth to live? Why?


In [34]:
query = 'how do i make chocolate cake?'
run_query(query)

0.79: How do you add chocolate chips to cake mix?
0.55: How do you make baking soda?
0.53: What is the red velvet cake frosting made of?
0.53: How do you make a perfume out of Skittles?
0.51: Where can I found adorable baked cupcakes in Gold Coast?
0.47: What is the recipe for a spinach artichoke dip from the Cheesecake Factory?
0.46: Where can I get custom decorated cakes for any occasion at Gold Coast?
0.45: How do I make chili?
0.44: What are all the different types of dessert?
0.43: Where can I get highest quality, tastiest cupcakes across the Gold Coast?
